In [1]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import seaborn as sns
import geopy.distance

import matplotlib.pyplot as plt

import urllib.request
import json

In [3]:
db_name = "../database/e-commerce.db"

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    # YOUR CODE
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)
        


conn = create_connection(db_name)
conn

2.6.0


In [ ]:

    cur = conn.cursor()
#     the table wight_cm can become width_cm   --> order_delivered_carrier_date == order_delivered_customer_date
    cur.execute('''
                SELECT 
                order_item.item_id, 
                order_item.freight_value, 
                order_item.price, 
                weight_g,
                lenght_cm, 
                height_cm, 
                width_cm, 
                customer.zip_code, 
                seller.zip_code, 
                order_dataset.purchase_timestamp,
                order_dataset.order_estimated_delivery_date, 
                order_dataset.order_delivered_customer_date
                FROM product
                LEFT JOIN order_item ON product.id = order_item.product_id
                LEFT JOIN order_dataset ON order_item.order_id = order_dataset.order_id
                LEFT JOIN customer ON customer.id = order_dataset.customer_id          
                LEFT JOIN seller ON order_item.seller_id = seller.id                
                ''')
    
    res_cur = cur.fetchall()
    

In [ ]:
df_bdd = pd.DataFrame(res_cur, columns=["nb_product", "prix_de_livraison", "product_price", "weight", "lenght", "height", "width", "zip_code_customer", "zip_code_seller", "date_purchase", "date_delevery_estimate", "date_delivery"])
df_bdd

In [ ]:
cur = conn.cursor()
#     the table wight_cm can become width_cm   --> order_delivered_carrier_date == order_delivered_customer_date
cur.execute('''
            SELECT 
            zip_code, latitude, longitude
            FROM geolocation
            ''')

res_cur = cur.fetchall()

df_geoloc = pd.DataFrame(res_cur, columns=["zip_code", "latitude", "longitude"])
df_geoloc


In [ ]:
df_bdd = df_bdd.merge(df_geoloc.drop_duplicates(['zip_code']), left_on='zip_code_customer', right_on='zip_code', how="left", suffixes = ("","_customer"))
df_bdd

In [ ]:
df_bdd = df_bdd.merge(df_geoloc.drop_duplicates(['zip_code']), left_on='zip_code_seller', right_on='zip_code', how="left", suffixes = ("","_seller"))
df_bdd

In [ ]:
df_bdd = df_bdd.rename({'latitude': 'latitude_customer', 'longitude': 'longitude_customer'}, axis=1)
df_bdd

## pour trouver la distance entre les geolocalisation nous tester plusieurs solutions.
### Solution 1:

In [ ]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

lat1 = radians(-22.2569861454338)
lon1 = radians(-45.6968690671541)
lat2 = radians(-22.4733036420427)
lon2 = radians(-45.6139087768263)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")


### Solution 2:
### nous decidons de garder d'utiliser cette methode car plus simple et lisible.

In [ ]:
coords_1 = (-22.2569861454338, -45.6968690671541)
coords_2 = (-22.4733036420427, -45.6139087768263)

print(geopy.distance.geodesic(coords_1, coords_2).km)

In [ ]:
distance = {"distance" : []
           }


for latitude_customer, longitude_customer, latitude_seller, longitude_seller in df_bdd[["latitude_customer","longitude_customer", "latitude_seller", "longitude_seller"]].to_numpy():
    coords_1 = (float(latitude_customer), float(longitude_customer))
    coords_2 = (float(latitude_seller), float(longitude_seller))
    try:
        res = geopy.distance.geodesic(coords_1, coords_2).km
        distance["distance"].append(res)
    except:
        distance["distance"].append(np.nan)
        print("error")
    
df_bdd["distance"] = distance["distance"]
df_bdd

# #     distance.append(geopy.distance.geodesic(coords_1, coords_2).km)
# for longitude, latitude in geoloc.items():
#     if longitude != 'nan' or latitude != 'nan':
#         print(longitude, latitude)




### on verifie que les valeurs inserer avec geopy sont bonne

In [ ]:
print(geopy.distance.geodesic((-22.331249, -46.922580), (-23.537922, -46.477696)).km)
print(geopy.distance.geodesic((-22.915371, -43.242996), (-23.468441, -46.720688)).km)

In [ ]:
for date in df_bdd[["date_purchase", "date_delevery_estimate", "date_delivery"]]:

In [ ]:
# calculate the correlation matrix
corr = df_bdd[["nb_product", "prix_de_livraison", "product_price", "weight", "lenght", "height", "width", "distance"]].corr()

# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns, annot=True)